In [1]:
# refresh_standings_silver.py
import statsapi
import pandas as pd
import requests
from datetime import date, datetime
from common import get_conn, log

def main():
    today = date.today().isoformat()
    log.info(f"Refreshing standings_silver for {today}")

    rows = []
    for league_id in [103, 104]:  # 103=AL, 104=NL
        data = statsapi.get("standings", {"leagueId": league_id, "season": 2025, "date": today})
        for rec in data.get("records", []):
            league = rec.get("league", {}) or {}
            division = rec.get("division", {}) or {}
            for tr in rec.get("teamRecords", []):
                team = tr.get("team", {}) or {}
                now = datetime.now().isoformat()
                rows.append({
                    "season": 2025,
                    "standings_date": today,
                    "league_id": league.get("id"),
                    "division_id": division.get("id"),
                    "team_id": team.get("id"),
                    "team_name": team.get("name"),
                    "wins": tr.get("wins"),
                    "losses": tr.get("losses"),
                    "pct": tr.get("winningPercentage"),
                    "games_back": tr.get("gamesBack"),
                    "wc_games_back": tr.get("wildCardGamesBack"),
                    "division_leader": tr.get("divisionLeader"),
                    "created_at": now,
                    "last_updated": now,
                })

    df = pd.DataFrame(rows)

    # Add league/division names
    leagues = requests.get("https://statsapi.mlb.com/api/v1/leagues").json()["leagues"]
    df_leagues = pd.DataFrame([{"league_id": l["id"], "league_name": l["name"]} for l in leagues])
    divs = requests.get("https://statsapi.mlb.com/api/v1/divisions").json()["divisions"]
    df_divs = pd.DataFrame([{"division_id": d["id"], "division_name": d["name"]} for d in divs])

    df = df.merge(df_leagues, on="league_id", how="left")
    df = df.merge(df_divs, on="division_id", how="left")

    log.info(f"Built {len(df)} rows")

    with get_conn() as conn:
        df.to_sql("standings_silver", conn, if_exists="replace", index=False)

    log.info("✅ standings_silver refresh complete")

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'common'